<a href="https://colab.research.google.com/github/bmill42/streaming-data/blob/main/Merging_listening_data_and_API_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Install spotipy, import the libraries we need, and set up the credentials to get data from the Spotify API. **Make sure to insert your own `client_id` and `client_secret`.**

Then load the listening data. Again, replace the filepath with the path to your own data.

In [ ]:
!pip install Spotipy

In [ ]:
import pandas as pd
from google.colab import drive
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

drive.mount('/content/drive')

client_id = ''
client_secret = ''

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
df = pd.read_json('/content/drive/MyDrive/COMPFOR 304/Data/BAM - Streaming_History_Audio_2013-2024.json')

# Getting audio features

I'm testing this on a subset of the rows using `head()` so I don't request 14k+ tracks' audio features.

The `uri_subset` variable stores a list of all the Spotify URIs for the tracks in my dataframe.

In [ ]:
tracks_subset = df.head()
uri_subset = tracks_subset.spotify_track_uri
uri_subset

Now we just ask for the audio features from the API and put them in a dataframe, exactly like in the original API examples.

In [ ]:
ft = sp.audio_features(uri_subset)
ft_df = pd.DataFrame(ft)
ft_df

# Merging audio features with listening data

Now we **merge** the features with the listening data by asking pandas to match up the URIs in our original data with the URIs in the audio features that we downloaded.

In [ ]:
subset_merged = pd.merge(tracks_subset, ft_df, left_on='spotify_track_uri', right_on='uri')

Looking at the columns of the resulting table, we can see that the original columns from the listening data (timestamp, track and artist name, and so on) have been augmented with the audio features, like 'danceability' and 'speechiness'.

In [ ]:
subset_merged.columns

There are still a lot of unneccessary columns here, so it's typically going to be best to reduce the table down to just the ones we care about.

In [ ]:
subset_merged[['ts', 'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'reason_end', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms']]